In [1]:
from GameVariants import PlayersGame
from Deck import Deck, Card
from players.RandomPlayer import RandomPlayer
from players.LodoPlayer import LodoPlayer
from PlayerNode import goto_player
import numpy as np
import math

# Analysis of game statistics (given random players)

the question to anser are:
- Who wins the most?
- Who loses the most?
- Is there a best strategy for predictions?
- Is there a best strategy for playing cards?
- Are the first or last player of a round in advantage?
- What is the right number to predict with n player and c cards?

## Check if there is a connection between winning and starting first or last

In [2]:
class FixedGame(PlayersGame):
    "A game where players never change position, 0 always first, 4 always last"
    
    def play(self, verbose=0):
        """ verbose: 0 no output, 1 only game stats, 2 output for human interaction, 3 full output"""
        self.verbose = verbose
        for round in range(self.max_rounds):

            hand_size = 5 - ((round+self.starting_round) % 5)

            if self.verbose >= 2:
                print(f"\n--- Round {round+1}: {hand_size} card(s) ---")

            if hand_size > 1:
                self.dealHands(hand_size)
                self.makePredictions(round, hand_size)

                # play cards
                for turn in range(hand_size):
                    self.first_in_turn[round, self.player_node.player.id] += 1
                    self.last_in_turn[round, goto_player(
                        self.player_node, (self.player_node.player.id+len(self.players)-1) % len(self.players)).player.id] += 1
                    if self.verbose >= 2:
                        print(f"--- Turn {turn+1} ---")

                    played_cards = self.playCards()
                    self.determineCatches(round, played_cards)
            else:
                visible_cards = []
                for p in self.players:
                    hand = self.deck.draw_hand(1)[0]
                    visible_cards.append(hand)
                    if self.verbose >= 3:
                        print(f"Player {p.id} hand: [{hand.strc()}]")

                self.makePredictions(round, hand_size, visible_cards)
                played_cards = {}
                for p in self.players:
                    played_cards[p.id] = visible_cards[p.id]
                self.determineCatches(round, played_cards)

                if self.verbose == 2:
                    print(f"All cards: [{', '.join([card.strc() for card in visible_cards])}]")

            if self.verbose >= 1:
                print(" ")

            removed_players = self.checkForErrors(round)
            if removed_players:
                if len(self.players) == 1 and self.verbose >= 1:
                    print(f"Winner: {self.players[0].id}")
                if len(self.players) <= 1:
                    if self.verbose >= 1:
                        print("Game Over")
                    break

            # self.current_player = self.current_player.next
            self.deck = Deck()

    def determineCatches(self, round, played_cards):
        ace = False
        for id, card in played_cards.items():
            if card.rank == '1' and card.suit == 'Denari':
                if self.predictions[round, id] > self.catches[round, id]:
                    ace = True
                    self.catches[round, id] += 1
                    # self.player_node = goto_player(self.player_node, id)
                    if self.verbose >= 2:
                        print(f"Player {id} takes")
                else:
                    # lowest possible card
                    played_cards[id] = Card('Bastoni', '1')

        if not ace:
            max = None
            max_idx = None
            for idx, card in played_cards.items():
                if max is None or card > max:
                    max = card
                    max_idx = idx
            self.catches[round, max_idx] += 1
            # self.player_node = goto_player(self.player_node, max_idx)
            if self.verbose >= 2:
                print(f"Player {max_idx} takes")


In [19]:
reps = 10000

for r in range(5):
    print(f"--- Round with {5-r} cards: ---")

    print("With random players:")
    players = [RandomPlayer(0), RandomPlayer(1), RandomPlayer(2), RandomPlayer(3)]
    stat_winners = np.zeros((reps, len(players)))

    for t in range(reps):
        game = FixedGame(players, r=1, e=-1, s=r)
        game.play(verbose=0)
        stats = game.return_stats()
        stat_winners[t, :] = stats["winners"]

    for i in range(len(players)):
        wins = np.sum(stat_winners[:, i] == 1)
        print(f"{i+1}° Player: Total wins: {wins}")


    print("With heuristic players:")
    players = [LodoPlayer(0), LodoPlayer(1), LodoPlayer(2), LodoPlayer(3)]
    stat_winners = np.zeros((reps, len(players)))

    for t in range(reps):
        game = FixedGame(players, r=1, e=-1, s=r)
        game.play(verbose=0)
        stats = game.return_stats()
        stat_winners[t, :] = stats["winners"]

    for i in range(len(players)):
        wins = np.sum(stat_winners[:, i] == 1)
        print(f"{i+1}° Player: Total wins: {wins}")

--- Round with 5 cards: ---
With random players:
1° Player: Total wins: 3807
2° Player: Total wins: 3767
3° Player: Total wins: 3736
4° Player: Total wins: 3799
With heuristic players:
1° Player: Total wins: 5314
2° Player: Total wins: 5206
3° Player: Total wins: 5278
4° Player: Total wins: 4712
--- Round with 4 cards: ---
With random players:
1° Player: Total wins: 3977
2° Player: Total wins: 3989
3° Player: Total wins: 3981
4° Player: Total wins: 3927
With heuristic players:
1° Player: Total wins: 5633
2° Player: Total wins: 5686
3° Player: Total wins: 5684
4° Player: Total wins: 4862
--- Round with 3 cards: ---
With random players:
1° Player: Total wins: 4343
2° Player: Total wins: 4247
3° Player: Total wins: 4303
4° Player: Total wins: 4193
With heuristic players:
1° Player: Total wins: 6303
2° Player: Total wins: 6313
3° Player: Total wins: 6331
4° Player: Total wins: 5078
--- Round with 2 cards: ---
With random players:
1° Player: Total wins: 4705
2° Player: Total wins: 4716
3° P

We see that with a heuristic, playing as last doesn't seems favorable.
Random players games are used as controls, without a strategy, going first or last should not change the outcome.

## Define the best theoretical strategy

### for 1 card

The only certain information a player has are the cards he can see.
This means that there is no advantage based on the starting position.
With this only, the best strategy is:
guess 0 if your card is probably not the highest => the highest of the card you see is above the avg. of the cards left (40-the cards of the other players)
guess 1 otherwise

<br>
When all players have optimal strategy, the predicitona of other players become relevant too, especially the one from the last players.
A prediction of 1 means that they belive they have a fair chance of taking => there isn't a card higher that the average.

In [8]:
reps = 10000

for n in range(1,8):
    print(f"--- With {n+1} players: ---")

    players = [RandomPlayer(id) for id in range(n)]
    players.append(LodoPlayer(n))
    stat_winners = np.zeros(n)

    for t in range(reps):
        game = PlayersGame(players, r=1, e=-1, s=4)
        game.play(verbose=0)
        stats = game.return_stats()
        for i in range(n):
            stat_winners[i] += stats["winners"][i]
    print(f"Heuristic wins: {int(stat_winners[-1])} out of {reps} games")


--- With 2 players: ---
Heuristic wins: 6300 out of 10000 games
--- With 3 players: ---
Heuristic wins: 5585 out of 10000 games
--- With 4 players: ---
Heuristic wins: 5366 out of 10000 games
--- With 5 players: ---
Heuristic wins: 5285 out of 10000 games
--- With 6 players: ---
Heuristic wins: 5207 out of 10000 games
--- With 7 players: ---
Heuristic wins: 5082 out of 10000 games
--- With 8 players: ---
Heuristic wins: 5055 out of 10000 games


This strategy consistently wins against a random binary choice even with a high number of players.

### 2 cards